Define a tiny example (4 frames) with reduced distributions:
- dirichlet over operations: flat
- hmm: 3 states, 2 units 
- gmm: 1 1d component



In [2]:
import numpy as np
import amdtk
import subprocess
import scipy
from scipy.stats import norm 


alphas = np.array([1,1,1,2,1])
# we need n_bottom_plus * n_hmm_states * n_components gaussians
# so that's 2*3*1 = 6 gaussians 
# 
mus = np.array([-3,-2,-1,1,2,3])
# let's make all variances same
sigmas = np.array([1,1,1,1,1,1])

t = np.array([[.5, .5, 0,0,0,0],
 [0,.5,.5,0,0,0,0],
 [0,0,.5,.5,0,0,0],
 [0,0,0,.5,.5,0,0],
 [.5,0,0,0,0,0,.5]])

data = np.array([-3,-1,2,3])
tops = np.array([0,1])

# state_llh is (n_frames, n_states * n_plus)
# so that's (4, 6) 
state_llh = np.log(np.array([[norm.pdf(x, mus[i], sigmas[i]) for i in range(6)] for x in data]))
print(np.round(np.exp(state_llh),4
              ))

INSIDE INIT
SUCCESSFUL IMPORT IN INIT
[[  3.98900000e-01   2.42000000e-01   5.40000000e-02   1.00000000e-04
    0.00000000e+00   0.00000000e+00]
 [  5.40000000e-02   2.42000000e-01   3.98900000e-01   5.40000000e-02
    4.40000000e-03   1.00000000e-04]
 [  0.00000000e+00   1.00000000e-04   4.40000000e-03   2.42000000e-01
    3.98900000e-01   2.42000000e-01]
 [  0.00000000e+00   0.00000000e+00   1.00000000e-04   5.40000000e-02
    2.42000000e-01   3.98900000e-01]]


In [3]:

model = amdtk.PhoneLoopNoisyChannel.create(
    n_units=2,  # number of acoustic units
    n_states=3,   # number of states per unit
    n_comp_per_state=1,   # number of Gaussians per emission
    n_top_units=2, # size of top PLU alphabet
    mean=np.zeros_like(mus), 
    var=np.ones_like(sigmas),
    max_slip_factor=.05
    #concentration=conc
)

log_prob_it = model.op_latent_posteriors[tops[0]].grad_log_partition[0]
log_prob_ib = model.op_latent_posteriors[tops[0]].grad_log_partition[1:model.n_units+1]
log_prob_sub = model.op_latent_posteriors[tops[0]].grad_log_partition[model.n_units+1:]
print("prob_it", np.exp(log_prob_it))
print("prob_ib", np.exp(log_prob_ib))
print("prob_sub", np.exp(log_prob_sub))



# ((frame_index, hmm_state, plu_bottom_type, plu_bottom_index, edit_op, plu_top_index), p)
starts = model.generate_start_items(tops, state_llh)
for tup, prob in starts:
    print(tup, np.exp(prob), prob)





    

[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]
[-2.08333333 -2.08333333 -2.08333333 -2.08333333 -2.08333333]
prob_it 0.124514471444
prob_ib [ 0.12451447  0.12451447]
prob_sub [ 0.12451447  0.12451447]
(0, 0, 0, 0, 0, -1) 0.0496740871809 -3.00227186654
(0, 0, 1, 0, 0, -1) 1.66637998244e-05 -11.0022718665
(-1, 2, 0, -1, 1, 0) 0.124514471444 -2.08333333333
(0, 0, 0, 0, 2, 0) 0.0496740871809 -3.00227186654
(0, 0, 1, 0, 2, 0) 1.66637998244e-05 -11.0022718665


In [4]:
# next_states(self, current_state, plu_tops, state_llh, max_slip, frames_per_top, log05, logging):
all_nexts = []
for i, state in enumerate(starts):
    nexts = model.next_states(state, tops, state_llh,  3, .05,np.log(.5), False)
    all_nexts.extend(nexts)
    print(nexts)
    print("for state", i)

[((1, 0, 0, 0, 3, -1), -6.6143575803026238), ((1, 1, 0, 0, 3, -1), -5.1143575803026238)]
for state 0
[((1, 0, 1, 0, 3, -1), -14.614357580302624), ((1, 1, 1, 0, 3, -1), -17.11435758030262)]
for state 1
[((0, 0, 0, 0, 0, 0), -5.7787523804312837), ((0, 0, 1, 0, 0, 0), -13.778752380431284), ((-1, 2, 0, -1, 1, 1), -4.1666666666666661), ((0, 0, 0, 0, 2, 1), -5.7787523804312837), ((0, 0, 1, 0, 2, 1), -13.778752380431284)]
for state 2
[((1, 0, 0, 0, 3, 0), -6.6143575803026238), ((1, 1, 0, 0, 3, 0), -5.1143575803026238)]
for state 3
[((1, 0, 1, 0, 3, 0), -14.614357580302624), ((1, 1, 1, 0, 3, 0), -17.11435758030262)]
for state 4


In [5]:

print(-3.00227186654 + np.log(.5) + state_llh[1,0])
print(-3.00227186654 + np.log(.5) + state_llh[1,1])


-6.6143575803
-5.1143575803


In [6]:
prev = -2.08333333333
print(prev + np.log(.5) + log_prob_ib[0] + state_llh[0,0])
print(prev + np.log(.5) + log_prob_ib[1] + state_llh[0,3])
print(prev + log_prob_it)
print(prev + np.log(.5) + log_prob_sub[0] + state_llh[0,0])
print(prev + np.log(.5) + log_prob_sub[1] + state_llh[0,3])


-5.77875238043
-13.7787523804
-4.16666666666
-5.77875238043
-13.7787523804


In [12]:
all_nexts2 = []
for state in all_nexts:
    print("for state :{}".format(state))
    try:
        nexts2 = model.next_states(state, tops, state_llh,  2, 1000,np.log(.5), False)
    except IndexError:
        continue
    print(nexts2)
    all_nexts2.extend(nexts2)

for state :((1, 0, 0, 0, 3, -1), -6.6143575803026238)
[((2, 0, 0, 0, 3, -1), -20.726443294067241), ((2, 1, 0, 0, 3, -1), -16.226443294067241)]
for state :((1, 1, 0, 0, 3, -1), -5.1143575803026238)
[((2, 1, 0, 0, 3, -1), -14.726443294067241), ((2, 2, 0, 0, 3, -1), -11.226443294067241)]
for state :((1, 0, 1, 0, 3, -1), -14.614357580302624)
[((2, 0, 1, 0, 3, -1), -16.726443294067241), ((2, 1, 1, 0, 3, -1), -16.226443294067241)]
for state :((1, 1, 1, 0, 3, -1), -17.11435758030262)
[((2, 1, 1, 0, 3, -1), -18.726443294067238), ((2, 2, 1, 0, 3, -1), -19.226443294067238)]
for state :((0, 0, 0, 0, 0, 0), -5.7787523804312837)
[((1, 0, 0, 0, 3, 0), -9.3908380941959013), ((1, 1, 0, 0, 3, 0), -7.8908380941959013)]
for state :((0, 0, 1, 0, 0, 0), -13.778752380431284)
[((1, 0, 1, 0, 3, 0), -17.3908380941959), ((1, 1, 1, 0, 3, 0), -19.8908380941959)]
for state :((-1, 2, 0, -1, 1, 1), -4.1666666666666661)
[]
for state :((0, 0, 0, 0, 2, 1), -5.7787523804312837)
[((1, 0, 0, 0, 3, 1), -9.3908380941959013)

In [8]:
all_nexts3 = []
for state in all_nexts2:
    print("for state :{}".format(state))
    try:
        nexts3 = model.next_states(state, tops, state_llh,  2, .05,np.log(.5), False)
    except IndexError:
        continue
    print(nexts3)
    all_nexts3.extend(nexts3)

for state :((2, 0, 0, 0, 3, -1), -20.726443294067241)
[((3, 0, 0, 0, 3, -1), -40.338529007831866), ((3, 1, 0, 0, 3, -1), -34.838529007831859)]
for state :((2, 1, 0, 0, 3, -1), -16.226443294067241)
[((3, 1, 0, 0, 3, -1), -30.338529007831855), ((3, 2, 0, 0, 3, -1), -25.838529007831855)]
for state :((2, 1, 0, 0, 3, -1), -14.726443294067241)
[((3, 1, 0, 0, 3, -1), -28.838529007831855), ((3, 2, 0, 0, 3, -1), -24.338529007831855)]
for state :((2, 2, 0, 0, 3, -1), -11.226443294067241)
[((3, 0, 0, 1, 0, -1), -inf), ((3, 0, 1, 1, 0, -1), -15.937141296499968), ((2, 2, 0, 0, 1, 0), -12.325055582735351), ((3, 0, 0, 1, 2, 0), -inf), ((3, 0, 1, 1, 2, 0), -15.937141296499968), ((3, 2, 0, 0, 3, -1), -20.838529007831855)]
for state :((2, 0, 1, 0, 3, -1), -16.726443294067241)
[((3, 0, 1, 0, 3, -1), -20.338529007831859), ((3, 1, 1, 0, 3, -1), -18.838529007831859)]
for state :((2, 1, 1, 0, 3, -1), -16.226443294067241)
[((3, 1, 1, 0, 3, -1), -18.338529007831859), ((3, 2, 1, 0, 3, -1), -17.838529007831859)]

In [13]:
all_nexts4 = []
for state in all_nexts3:
    print("for state :{}".format(state))
    try:
        nexts4 = model.next_states(state, tops, state_llh,  2, .05,np.log(.5), False)
    except IndexError:
        continue
    print(nexts4)
    all_nexts4.extend(nexts4)

for state :((3, 0, 0, 0, 3, -1), -40.338529007831866)
[]
for state :((3, 1, 0, 0, 3, -1), -34.838529007831859)
[]
for state :((3, 1, 0, 0, 3, -1), -30.338529007831855)
[]
for state :((3, 2, 0, 0, 3, -1), -25.838529007831855)
[((3, 2, 0, 0, 1, 0), -26.937141296499966)]
for state :((3, 1, 0, 0, 3, -1), -28.838529007831855)
[]
for state :((3, 2, 0, 0, 3, -1), -24.338529007831855)
[((3, 2, 0, 0, 1, 0), -25.437141296499966)]
for state :((3, 0, 0, 1, 0, -1), -inf)
[]
for state :((3, 0, 1, 1, 0, -1), -15.937141296499968)
[]
for state :((2, 2, 0, 0, 1, 0), -12.325055582735351)
[((3, 0, 0, 1, 0, 0), -inf), ((3, 0, 1, 1, 0, 0), -17.035753585168077), ((2, 2, 0, 0, 1, 1), -13.42366787140346), ((3, 0, 0, 1, 2, 1), -inf), ((3, 0, 1, 1, 2, 1), -17.035753585168077)]
for state :((3, 0, 0, 1, 2, 0), -inf)
[]
for state :((3, 0, 1, 1, 2, 0), -15.937141296499968)
[]
for state :((3, 2, 0, 0, 3, -1), -20.838529007831855)
[((3, 2, 0, 0, 1, 0), -21.937141296499966)]
for state :((3, 0, 1, 0, 3, -1), -20.3385290

In [14]:
all_nexts5 = []
for state in all_nexts4:
    print("for state :{}".format(state))
    try:
        nexts5 = model.next_states(state, tops, state_llh,  2, .05,np.log(.5), False)
    except IndexError:
        continue
    print(nexts5)
    all_nexts5.extend(nexts5)

for state :((3, 2, 0, 0, 1, 0), -26.937141296499966)
[((3, 2, 0, 0, 1, 1), -28.035753585168077)]
for state :((3, 2, 0, 0, 1, 0), -25.437141296499966)
[((3, 2, 0, 0, 1, 1), -26.535753585168077)]
for state :((3, 0, 0, 1, 0, 0), -inf)
[]
for state :((3, 0, 1, 1, 0, 0), -17.035753585168077)
[]
for state :((2, 2, 0, 0, 1, 1), -13.42366787140346)
[]
for state :((3, 0, 0, 1, 2, 1), -inf)
[]
for state :((3, 0, 1, 1, 2, 1), -17.035753585168077)
[]
for state :((3, 2, 0, 0, 1, 0), -21.937141296499966)
[((3, 2, 0, 0, 1, 1), -23.035753585168077)]
for state :((3, 2, 1, 0, 1, 0), -18.93714129649997)
[((3, 2, 1, 0, 1, 1), -20.035753585168081)]
for state :((3, 2, 1, 0, 1, 0), -21.437141296499966)
[((3, 2, 1, 0, 1, 1), -22.535753585168077)]
for state :((3, 0, 0, 1, 0, 0), -41.035753585168081)
[]
for state :((3, 0, 1, 1, 0, 0), -inf)
[]
for state :((2, 2, 1, 0, 1, 1), -21.42366787140346)
[]
for state :((3, 0, 0, 1, 2, 1), -41.035753585168081)
[]
for state :((3, 0, 1, 1, 2, 1), -inf)
[]
for state :((3, 2,

so far, forward is looking ok. Let's look at backwards

In [22]:

# def prev_states(self, current_state, plu_tops, state_llh, max_slip, frames_per_top, log05):

end_0 = model.generate_end_items(tops, state_llh, 4)

all_back_states_1 = []
for state in end_0:
    nexts1 = model.prev_states(state, tops, state_llh, 2, .05, np.log(0.5))
    all_back_states_1.extend(nexts1)
    print("for state {}".format(state))
    print(nexts1)

for state ((3, 2, 0, 0, 1, 1), 0.0)
[((3, 2, 0, 0, 1, 0), -1.0986122886681098), ((3, 2, 0, 0, 3, 0), -1.0986122886681098)]
for state ((3, 2, 0, 0, 3, 1), 0.0)
[((2, 2, 0, 0, 3, 1), -9.6120857137646176), ((2, 1, 0, 0, 3, 1), -9.6120857137646176)]
for state ((3, 2, 0, 1, 1, 1), 0.0)
[((3, 2, 0, 1, 1, 0), -1.0986122886681098), ((3, 2, 0, 1, 3, 0), -1.0986122886681098)]
for state ((3, 2, 0, 1, 3, 1), 0.0)
[((2, 2, 0, 1, 3, 1), -9.6120857137646176), ((2, 1, 0, 1, 3, 1), -9.6120857137646176)]
for state ((3, 2, 0, 2, 1, 1), 0.0)
[((3, 2, 0, 2, 1, 0), -1.0986122886681098), ((3, 2, 0, 2, 3, 0), -1.0986122886681098)]
for state ((3, 2, 0, 2, 3, 1), 0.0)
[((2, 2, 0, 2, 3, 1), -9.6120857137646176), ((2, 1, 0, 2, 3, 1), -9.6120857137646176)]
for state ((3, 2, 0, 3, 1, 1), 0.0)
[]
for state ((3, 2, 0, 3, 3, 1), 0.0)
[((2, 2, 0, 3, 3, 1), -9.6120857137646176), ((2, 1, 0, 3, 3, 1), -9.6120857137646176)]
for state ((3, 2, 0, 4, 1, 1), 0.0)
[]
for state ((3, 2, 0, 4, 3, 1), 0.0)
[((2, 2, 0, 4, 3, 1), -9.

In [25]:
print(log_prob_it )

-2.77648051389
